# Build data with raptor

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService
provider = ProviderService()

In [2]:
DATA_REPO = "BroDeadlines/TEST.edu_tdt_proposition_data"
# SPLIT = "TEST.basic_index_TDT_clean"
SPLIT = "INDEX.medium_index_TDT_clean"
test_dataset = load_dataset(DATA_REPO, split= SPLIT)
test_dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [3]:
rows = test_dataset.filter(lambda e: e['url'] == "https://tuvanhocduong.tdtu.edu.vn/News")
rows

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits', 'split', 'propositions', 'proposition_list'],
    num_rows: 144
})

## Update the splits

In [4]:
dataset = test_dataset.remove_columns(['split', 'splits', 'shards'])
dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'propositions', 'proposition_list'],
    num_rows: 344
})

In [5]:
from src.utils.text_utils import create_splitter, try_split_texts
from src.chain.proposition import create_proposition_chain

contents = dataset['content']
splitter = create_splitter(chunk_size=300, overlap=0, separators=[".", "\n"])
try_split_texts(texts=contents, splitter=splitter)

From 344 texts -> 1603
Distribution: Counter({1: 129, 5: 46, 2: 27, 4: 26, 6: 25, 3: 15, 8: 15, 7: 12, 9: 11, 10: 8, 11: 6, 12: 6, 13: 5, 27: 2, 40: 2, 17: 2, 15: 2, 14: 1, 20: 1, 21: 1, 24: 1, 28: 1})
== About the splits ==
The mean of the splits is: 4.659883720930233
The median of the splits is: 4.0
== About the splits length ==
The mean of the split length is: 236.15658140985653
The median of the split length is: 251.0


In [6]:
def create_split(row):
    docs = splitter.split_text(row['content'])
    return {**row, "splits": docs}
    
new_ds = dataset.map(create_split)
new_ds

Dataset({
    features: ['content', 'url', 'doc_id', 'propositions', 'proposition_list', 'splits'],
    num_rows: 344
})

In [7]:
a = new_ds[10]['splits']
len(a)

2

## Build RAPTOR tree

In [1]:
from src.rag.raptor import RAPTOR
raptor = RAPTOR(provider)

# vec_idx = "vec-raptor-basic_index_tdt_clean"
# text_idx = "text-raptor-basic_index_tdt_clean"

text_idx = "text-raptor-medium_index_tdt"
vec_idx = "vec-raptor-medium_index_tdt"

# text_idx = "text-raptor-medium_index_tdt"
# vec_idx = "vec-raptor-medium_index_tdt"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

ModuleNotFoundError: No module named 'src'

In [10]:
leaf_ids = []
leaf_texts = []

def collect_new_data(row):
    docs = row['splits']
    for d in docs:
        leaf_ids.append(row['doc_id'])
        leaf_texts.append(d)
    return row

a = new_ds.map(collect_new_data)
print(len(leaf_texts) ,"-", len(leaf_ids))
a

Map:   0%|          | 0/344 [00:00<?, ? examples/s]

1603 - 1603


Dataset({
    features: ['content', 'url', 'doc_id', 'propositions', 'proposition_list', 'splits'],
    num_rows: 344
})

In [11]:
results = raptor.recursive_embed_cluster_summarize(leaf_texts, leaf_ids, level=1, n_levels=3)

--Generated 264 clusters--
--Generated 58 clusters--
--Generated 10 clusters--


### Up to Hugging Face

In [27]:
import pandas as pd

def assign_level_id(raptor_tree):
    levels = raptor_tree.keys()
    for level in levels:
        raptor_tree[level][0]['level_id'] = f"tree_{level}"
        raptor_tree[level][1]['level_id'] = f"tree_{level}"
    return

def print_raptor_info(results):
    print(results.keys())
    for i in results.keys():
        print("=======")
        print(len(results[i]))
        print(results[i][0].keys())
        print("->",results[i][0].shape)
        print(results[i][0]['level_id'].unique())
        print(results[i][1].keys())
        print("->",results[i][1].shape)
    return

def flatten_tree(raptor_tree):
    clusters = []
    summarizes = []
    for level in raptor_tree.keys():
        clusters.append(raptor_tree[level][0])
        summarizes.append(raptor_tree[level][1])
    return pd.concat(clusters), pd.concat(summarizes)

In [28]:
# assign_level_id(results)
print_raptor_info(results)
# clusters, summarizes = flatten_tree(results)

dict_keys([1, 2, 3])
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (1603, 5)
['tree_1']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (264, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (264, 5)
['tree_2']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (58, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (58, 5)
['tree_3']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (10, 5)


In [19]:
from datasets import Dataset
cluster_dataset = Dataset.from_pandas(clusters)
print(cluster_dataset)
summerize_dataset = Dataset.from_pandas(summarizes)
summerize_dataset

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 1925
})


Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 332
})

In [20]:
CLUSTER_REPO = "BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data"
SUMMERIZE_REPO = "BroDeadlines/TEST.PART_SUMMERIZE.raptor.edu_tdt_data"
SPLIT = "TEST.medium_tdt_raptor"

In [21]:
cluster_dataset.push_to_hub(CLUSTER_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/513 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data/commit/cf890fb6073bec9afc6b67fa28defec4bf1f8a6e', commit_message='Upload dataset', commit_description='', oid='cf890fb6073bec9afc6b67fa28defec4bf1f8a6e', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
summerize_dataset.push_to_hub(SUMMERIZE_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/654 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.PART_SUMMERIZE.raptor.edu_tdt_data/commit/077f9bf847f38e45aebf89809c20db15b45e1d64', commit_message='Upload dataset', commit_description='', oid='077f9bf847f38e45aebf89809c20db15b45e1d64', pr_url=None, pr_revision=None, pr_num=None)

### Up to ES

In [29]:
print_raptor_info(results)


dict_keys([1, 2, 3])
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (1603, 5)
['tree_1']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (264, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (264, 5)
['tree_2']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (58, 5)
2
Index(['text', 'embd', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (58, 5)
['tree_3']
Index(['summaries', 'level', 'cluster', 'doc_ids', 'level_id'], dtype='object')
-> (10, 5)


In [23]:
from src.utils.type_utils import create_langchain_doc
# Initialize all_texts with leaf_texts
all_texts = [create_langchain_doc(txt, {"doc_id": id}) for txt, id in zip(leaf_texts, leaf_ids)]
all_texts[0]

Document(page_content='Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.', metadata={'doc_id': '573ca174-1b6d-11ef-a755-d38426455a06'})

In [32]:
1603 + 264 + 58 + 10

1935

In [24]:
len(all_texts)

1603

In [25]:

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    d_ids = results[level][1]["doc_ids"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend([create_langchain_doc(txt, {"doc_id": id}) for txt, id in zip(summaries, d_ids)])


In [31]:
len(all_texts)

1935

In [33]:
b = bm.add_documents(all_texts)

In [35]:
with open("bm_ids.txt", "w") as f:
    f.write("\n".join(b))

In [36]:
a = es.add_documents(all_texts)

In [37]:
with open("es_ids.txt", "w") as f:
    f.write("\n".join(a))

In [72]:
es.add_documents(all_texts)

['9fa72e72-fd64-4884-b53e-db4bb733dc5f',
 '6d987e23-848e-4459-b0fa-d0c0e100344d',
 '6e458c05-1c10-4c48-a512-6bab38abceb5',
 'd1382816-a86b-4986-90fb-47ceb6579e24',
 '85a355c6-1b7d-4c40-926b-3498807f80ab',
 '1e78e3b5-3c8a-4eb2-a3c4-7a55bc603c9a',
 '647cf3e2-2460-4a52-ad6f-50078c7d62ce',
 '4366da21-0ce1-4499-9376-17aa453b91c9',
 '46714ac8-cce6-40ac-a929-1350c6fb441f',
 'ba72db72-43c3-44f6-9afe-6703c03d4085',
 '16ea42fc-7445-4ccf-9fa0-9e80e31cbc5a',
 'e09c451d-d501-42dc-b54a-7d09d3996d28',
 'ecb9eb53-1014-4d0f-9bdb-e56c8e232bcb',
 '09a32930-e81d-4078-aa3a-795bc352508e',
 'c6009ebf-f340-47ee-82d6-df25ae8bc707',
 '10ad2171-5136-4c36-bb6e-cec729c036c2',
 '730802cd-8237-47f1-9bd9-8fc75c04c1f3',
 '2157f697-223c-47a3-b65c-e8aa6f10007b',
 '47381643-c416-42c2-9a5e-c75f8aa6ffff',
 'a1815ff3-ba25-4cc9-91ac-525f3a280273',
 'e670e988-2a2f-492c-8e75-8adebcfe482a',
 '86a4ca28-3051-462f-8e44-bd9b27a04fde',
 '80de9b5e-9c6a-4468-8d90-8732d5c0763d',
 '0bccdf8f-a753-4f12-9eb8-7413b67389cb',
 '37d01bf2-fcea-

In [73]:
bm.add_documents(all_texts)

['607baae7-5d53-4fc2-b0e8-cb5ce2ec8907',
 '527ce5d8-606d-4383-b05a-3266bf65e604',
 'b86d30d1-4673-4f37-8265-3fc00398ad4a',
 '0a175630-6557-45f2-a41f-dd5566d347e6',
 '926c01bf-5463-4381-9694-14b18386094a',
 '9bfbdd09-1a1e-42a1-9870-9b9fbc16742a',
 '3092a61b-2a78-4715-a4cc-cb001e95a018',
 'e62c729b-f63e-475d-9a9a-02c4372c1674',
 '9ba78eeb-7487-4679-9059-cdf25c15f507',
 '6e92f07f-819c-4f21-aa1c-d7b48ae39a4a',
 'c1de9a0d-7896-4f2f-a971-1f220bd3c3d3',
 '5b7d9828-c6f1-46be-8c7c-22046ef9b33e',
 'f2e9e8b5-b0a8-43c7-a1be-02eff71e45c3',
 'e419504a-2966-4c42-a384-a0a1b1aa4eab',
 'bf16690e-7af5-4633-9cb3-b71810fd87a7',
 '85c911bc-c4cc-4597-91a8-b74a806fa773',
 '90bdeaa7-b10e-4cc9-8779-b25d7f2790ba',
 '65d86756-1017-4be5-b150-580bb721cc5e',
 'f21727ab-051a-4cdc-a9ac-ee4fb2627f59',
 '9f0bec38-902d-406c-b716-0809681d8861',
 '0f93f264-9b2a-4244-b75c-0075ab2916b8',
 'ed617ef1-fed8-4f49-81d7-a5f264998879',
 '90c98915-3bfb-4298-b01d-6a523c3aac94',
 '96d4c5d8-c443-4ee4-9745-191781753388',
 '6ede84f4-41ef-

## Update shards

In [39]:
print(len(all_texts))
all_texts[0]

1935


Document(page_content='Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.', metadata={'doc_id': '573ca174-1b6d-11ef-a755-d38426455a06'})

In [48]:
def build_shards(all_data):
    shards_map = {}
    shards = {}
    for doc in all_data:
        d_id = doc.metadata['doc_id']
        # update new record
        if d_id not in shards_map:
            shards_map[d_id] = 1
            shards[d_id] = 1
        else:
            shards_map[d_id] += 1
            shards[d_id] += 1
        # update sub-tree
        if ";" not in d_id:
            continue
        keys = shards_map.keys()
        for k in keys:
            if k in d_id:
                shards_map[k] += 1
    return shards, shards_map

shards, hard_shards = build_shards(all_texts)
print(sum(shards.values()))
print(sum(hard_shards.values()))

1935
8810


In [47]:
print(cluster_dataset)
summerize_dataset

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 1925
})


Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 332
})

In [50]:
def update_shards(row):
    row['easy_shards'] = shards[row['doc_ids']]
    row['hard_shards'] = hard_shards[row['doc_ids']]
    return row
    
cluster_dataset_one = cluster_dataset.map(update_shards)
summerize_dataset_one = summerize_dataset.map(update_shards)

Map:   0%|          | 0/1925 [00:00<?, ? examples/s]

Map:   0%|          | 0/332 [00:00<?, ? examples/s]

In [52]:
summerize_dataset_one = summerize_dataset_one.remove_columns(['__index_level_0__'])
cluster_dataset_one = cluster_dataset_one.remove_columns(['__index_level_0__'])
print(cluster_dataset_one)
summerize_dataset_one

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', 'easy_shards', 'hard_shards'],
    num_rows: 1925
})


Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', 'easy_shards', 'hard_shards'],
    num_rows: 332
})

In [55]:
CLUSTER_REPO = "BroDeadlines/TEST.NEW.PART_CLUSTER.raptor.edu_tdt_data"
SUMMERIZE_REPO = "BroDeadlines/TEST.NEW.PART_SUMMERIZE.raptor.edu_tdt_data"
SPLIT = "TEST.medium_tdt_raptor"

cluster_dataset_one.push_to_hub(CLUSTER_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/556 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.NEW.PART_CLUSTER.raptor.edu_tdt_data/commit/1b87bbfa25f53ec27d7ac5284baeef337e3a2268', commit_message='Upload dataset', commit_description='', oid='1b87bbfa25f53ec27d7ac5284baeef337e3a2268', pr_url=None, pr_revision=None, pr_num=None)

In [56]:
summerize_dataset_one.push_to_hub(SUMMERIZE_REPO, split=SPLIT)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/549 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/TEST.NEW.PART_SUMMERIZE.raptor.edu_tdt_data/commit/5f3635160e0d1a5bbb4ac575e6bb60255e8532bc', commit_message='Upload dataset', commit_description='', oid='5f3635160e0d1a5bbb4ac575e6bb60255e8532bc', pr_url=None, pr_revision=None, pr_num=None)

In [2]:
SUM_REPO = "BroDeadlines/TEST.PART_SUMMERIZE.raptor.edu_tdt_data"
CLU_REPO = "BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data"
SPLIT = "TEST.basic_tdt_raptor"
sum_dataset = load_dataset(SUM_REPO, split= SPLIT)
sum_dataset

Generating TEST.basic_tdt_raptor split:   0%|          | 0/19 [00:00<?, ? examples/s]

Dataset({
    features: ['summaries', 'level', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 19
})

In [3]:
CLU_REPO = "BroDeadlines/TEST.PART_CLUSTER.raptor.edu_tdt_data"
SPLIT = "TEST.basic_tdt_raptor"
cluster_dataset = load_dataset(CLU_REPO, split= SPLIT)
cluster_dataset

Generating TEST.basic_tdt_raptor split:   0%|          | 0/117 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'embd', 'cluster', 'doc_ids', 'level_id', '__index_level_0__'],
    num_rows: 117
})

## Done